In [2]:
import argparse
import os

from filelock import FileLock
from tensorflow.keras.datasets import fashion_mnist

import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.integration.keras import TuneReportCallback


def train_mnist(config):
    # https://github.com/tensorflow/tensorflow/issues/32159
    import tensorflow as tf
    batch_size = 128
    num_classes = 10
    epochs = 12

    with FileLock(os.path.expanduser("~/.data.lock")):
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(config["hidden"], activation=config["activation"]),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(num_classes, activation="softmax")
    ])

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.SGD(
            lr=config["lr"], momentum=config["momentum"]),
        metrics=["accuracy"])

    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=0,
        validation_data=(x_test, y_test),
        callbacks=[TuneReportCallback({
            "mean_accuracy": "accuracy"
        })])


sched = AsyncHyperBandScheduler(time_attr="training_iteration", max_t=400, grace_period=20)

analysis = tune.run(
        train_mnist,
        name="exp",
        scheduler=sched,
        metric="mean_accuracy",
        mode="max",
        num_samples=10,
        resources_per_trial={
            "cpu": 2,
            "gpu": 0
        },
        config={
            "threads": 2,
            "lr": tune.uniform(0.001, 0.1),
            "momentum": tune.uniform(0.1, 0.9),
            "hidden": tune.randint(32, 512),
            "activation":tune.choice(['relu','tanh'])
        })
    
print("Best hyperparameters found were: ", analysis.best_config)


Trial name,status,loc,activation,hidden,lr,momentum
train_mnist_a8a77_00000,PENDING,,tanh,201,0.097084,0.276748
train_mnist_a8a77_00001,PENDING,,tanh,285,0.0214803,0.546068
train_mnist_a8a77_00002,PENDING,,tanh,149,0.046793,0.618789
train_mnist_a8a77_00003,PENDING,,tanh,471,0.0425301,0.119791
train_mnist_a8a77_00004,PENDING,,tanh,163,0.0995049,0.513953
train_mnist_a8a77_00005,PENDING,,tanh,83,0.0510857,0.740249
train_mnist_a8a77_00006,PENDING,,tanh,484,0.00604322,0.391449
train_mnist_a8a77_00007,PENDING,,tanh,278,0.025687,0.711537
train_mnist_a8a77_00008,PENDING,,tanh,477,0.0588453,0.628015
train_mnist_a8a77_00009,PENDING,,relu,134,0.0351733,0.164561


 pid=13260) 2022-02-28 20:36:07.156068: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
 pid=13260) 2022-02-28 20:36:07.156151: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Trial name,status,loc,activation,hidden,lr,momentum
train_mnist_a8a77_00000,RUNNING,127.0.0.1:13260,tanh,201,0.097084,0.276748
train_mnist_a8a77_00001,PENDING,,tanh,285,0.0214803,0.546068
train_mnist_a8a77_00002,PENDING,,tanh,149,0.046793,0.618789
train_mnist_a8a77_00003,PENDING,,tanh,471,0.0425301,0.119791
train_mnist_a8a77_00004,PENDING,,tanh,163,0.0995049,0.513953
train_mnist_a8a77_00005,PENDING,,tanh,83,0.0510857,0.740249
train_mnist_a8a77_00006,PENDING,,tanh,484,0.00604322,0.391449
train_mnist_a8a77_00007,PENDING,,tanh,278,0.025687,0.711537
train_mnist_a8a77_00008,PENDING,,tanh,477,0.0588453,0.628015
train_mnist_a8a77_00009,PENDING,,relu,134,0.0351733,0.164561


 pid=19760) 2022-02-28 20:36:13.799955: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
 pid=19760) 2022-02-28 20:36:13.800082: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
 pid=13260) 2022-02-28 20:36:13.922588: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
 pid=13260) 2022-02-28 20:36:13.923119: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-35S3A4G
 pid=13260) 2022-02-28 20:36:13.923325: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-35S3A4G
 pid=13260) 2022-02-28 20:36:13.954251: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following 

Trial name,status,loc,activation,hidden,lr,momentum
train_mnist_a8a77_00000,RUNNING,127.0.0.1:13260,tanh,201,0.097084,0.276748
train_mnist_a8a77_00001,RUNNING,127.0.0.1:19760,tanh,285,0.0214803,0.546068
train_mnist_a8a77_00002,PENDING,,tanh,149,0.046793,0.618789
train_mnist_a8a77_00003,PENDING,,tanh,471,0.0425301,0.119791
train_mnist_a8a77_00004,PENDING,,tanh,163,0.0995049,0.513953
train_mnist_a8a77_00005,PENDING,,tanh,83,0.0510857,0.740249
train_mnist_a8a77_00006,PENDING,,tanh,484,0.00604322,0.391449
train_mnist_a8a77_00007,PENDING,,tanh,278,0.025687,0.711537
train_mnist_a8a77_00008,PENDING,,tanh,477,0.0588453,0.628015
train_mnist_a8a77_00009,PENDING,,relu,134,0.0351733,0.164561


...

Result for train_mnist_a8a77_00008:
  date: 2022-02-28_20-42-45
  done: false
  experiment_id: c3ddca6ce32c4a60b71df88036230e69
  hostname: DESKTOP-35S3A4G
  iterations_since_restore: 11
  mean_accuracy: 0.8700500130653381
  node_ip: 127.0.0.1
  pid: 7708
  time_since_restore: 76.79690384864807
  time_this_iter_s: 4.0716612339019775
  time_total_s: 76.79690384864807
  timestamp: 1646077365
  timesteps_since_restore: 0
  training_iteration: 11
  trial_id: a8a77_00008
  


Trial name,status,loc,activation,hidden,lr,momentum,acc,iter,total time (s)
train_mnist_a8a77_00008,RUNNING,127.0.0.1:7708,tanh,477,0.0588453,0.628015,0.87005,11,76.7969
train_mnist_a8a77_00002,TERMINATED,127.0.0.1:12728,tanh,149,0.046793,0.618789,0.869733,12,56.0818
train_mnist_a8a77_00003,TERMINATED,127.0.0.1:16880,tanh,471,0.0425301,0.119791,0.860533,12,86.2825
train_mnist_a8a77_00004,TERMINATED,127.0.0.1:14208,tanh,163,0.0995049,0.513953,0.872567,12,54.3838
train_mnist_a8a77_00005,TERMINATED,127.0.0.1:5292,tanh,83,0.0510857,0.740249,0.8713,12,64.0592
train_mnist_a8a77_00006,TERMINATED,127.0.0.1:23696,tanh,484,0.00604322,0.391449,0.843433,12,92.3942
train_mnist_a8a77_00007,TERMINATED,127.0.0.1:21360,tanh,278,0.025687,0.711537,0.8666,12,59.8376
train_mnist_a8a77_00009,TERMINATED,127.0.0.1:8792,relu,134,0.0351733,0.164561,0.869517,12,55.673
train_mnist_a8a77_00000,ERROR,127.0.0.1:13260,tanh,201,0.097084,0.276748,0.854667,5,26.8886
train_mnist_a8a77_00001,ERROR,127.0.0.1:19760,tanh,285,0.0214803,0.546068,0.76925,1,17.5049


Result for train_mnist_a8a77_00008:
  date: 2022-02-28_20-42-48
  done: true
  experiment_id: c3ddca6ce32c4a60b71df88036230e69
  experiment_tag: 8_activation=tanh,hidden=477,lr=0.058845,momentum=0.62802
  hostname: DESKTOP-35S3A4G
  iterations_since_restore: 12
  mean_accuracy: 0.8706666827201843
  node_ip: 127.0.0.1
  pid: 7708
  time_since_restore: 80.6506917476654
  time_this_iter_s: 3.853787899017334
  time_total_s: 80.6506917476654
  timestamp: 1646077368
  timesteps_since_restore: 0
  training_iteration: 12
  trial_id: a8a77_00008
  


Trial name,status,loc,activation,hidden,lr,momentum,acc,iter,total time (s)
train_mnist_a8a77_00002,TERMINATED,127.0.0.1:12728,tanh,149,0.046793,0.618789,0.869733,12,56.0818
train_mnist_a8a77_00003,TERMINATED,127.0.0.1:16880,tanh,471,0.0425301,0.119791,0.860533,12,86.2825
train_mnist_a8a77_00004,TERMINATED,127.0.0.1:14208,tanh,163,0.0995049,0.513953,0.872567,12,54.3838
train_mnist_a8a77_00005,TERMINATED,127.0.0.1:5292,tanh,83,0.0510857,0.740249,0.8713,12,64.0592
train_mnist_a8a77_00006,TERMINATED,127.0.0.1:23696,tanh,484,0.00604322,0.391449,0.843433,12,92.3942
train_mnist_a8a77_00007,TERMINATED,127.0.0.1:21360,tanh,278,0.025687,0.711537,0.8666,12,59.8376
train_mnist_a8a77_00008,TERMINATED,127.0.0.1:7708,tanh,477,0.0588453,0.628015,0.870667,12,80.6507
train_mnist_a8a77_00009,TERMINATED,127.0.0.1:8792,relu,134,0.0351733,0.164561,0.869517,12,55.673
train_mnist_a8a77_00000,ERROR,127.0.0.1:13260,tanh,201,0.097084,0.276748,0.854667,5,26.8886
train_mnist_a8a77_00001,ERROR,127.0.0.1:19760,tanh,285,0.0214803,0.546068,0.76925,1,17.5049


TuneError: ('Trials did not complete', [train_mnist_a8a77_00000, train_mnist_a8a77_00001])